# Intermediate Python
### Patrick Loeber, python-engineer.com
### https://www.youtube.com/watch?v=HGOBQPFzWKo
(2:42:20)
September 17, 2022

## JSON: short for Javascript Object Notation
A lightweight data format used for data exchange, used heavily in web applications. Python has a built in JSON module to make it easy to work with.

In [1]:
# See example.json for the json file
# JSON data looks very similar to a dictionary with key-value pairs
# It can takes, strings, booleans, etc
# More info on python-engineer.com


In [2]:
import json

In [3]:
# We have a python dictionary and want to convert it to a JSON format, a process known as serialization, or encoding

person = {
    'name': 'Bob',
    'age': 42,
    'job': 'dev',
    'salary': 100000,
    'married': True,
    'children': ['Alice', 'John'],
    'pets': None,
    'cars': [
        {'model': 'BMW 230', 'mpg': 27.5},
        {'model': 'Ford Edge', 'mpg': 24.1}
    ]
}

In [6]:
# This will dump our object to a JSON string (s means string)
# Indentation helps with visual format
# different separators can be set as well:  separators='; ', '= '
# keys can be sorted alphabetically

personJSON = json.dumps(person, indent=4, sort_keys=True)

In [8]:
print(personJSON)
# printed, you can see that it is now in JSON format, for example
# false is now lowercase

{
    "age": 42,
    "cars": [
        {
            "model": "BMW 230",
            "mpg": 27.5
        },
        {
            "model": "Ford Edge",
            "mpg": 24.1
        }
    ],
    "children": [
        "Alice",
        "John"
    ],
    "job": "dev",
    "married": true,
    "name": "Bob",
    "pets": null,
    "salary": 100000
}


In [10]:
# Can also be dumped into a file, auto created
# This creates person.json
with open('person.json', 'w') as file:
    json.dump(person, file, indent=4)

### CONVERTING BACK TO PYTHON:
de-serialization, decoding

In [12]:
# Loading the personJSON object (a string) back as a python dict
person2 = json.loads(personJSON)

print(person2)

{'age': 42, 'cars': [{'model': 'BMW 230', 'mpg': 27.5}, {'model': 'Ford Edge', 'mpg': 24.1}], 'children': ['Alice', 'John'], 'job': 'dev', 'married': True, 'name': 'Bob', 'pets': None, 'salary': 100000}


In [13]:
# Loading back into python from a file
with open('person.json', 'r') as file:
    person3 = json.load(file)

print(person3)

{'name': 'Bob', 'age': 42, 'job': 'dev', 'salary': 100000, 'married': True, 'children': ['Alice', 'John'], 'pets': None, 'cars': [{'model': 'BMW 230', 'mpg': 27.5}, {'model': 'Ford Edge', 'mpg': 24.1}]}


### Working this way with a custom class:
Encoding custom objects into JSON

In [16]:
class User:
    def __init__(self, name, age):
        self.name = name
        self.age = age


user = User('Max', 27)

# The following creates a type error, which means we must write
# a short custom encoding function.

# userJSON = json.dumps(user)

In [21]:
def encode_user(object):
    # check if an object is an instance of a class
    # if so, we will return the information formatted as a
    # dictionary as shown in the return

    if isinstance(object, User):
        return {'name': object.name, 'age': object.age,
                # We can also include the following to indicate
                # if the object is from a class
                object.__class__.__name__: True}

    else:
        raise TypeError('Object of type User is not JSON serializable')

In [22]:
# now, we pass the new function to the conversion json.dumps
# and it will know how to deal with the data

userJSON = json.dumps(user, default=encode_user)

In [23]:
print(userJSON)

{"name": "Max", "age": 27, "User": true}


### We can also implement a custon JSON encoder:

In [27]:
from json import JSONEncoder

In [30]:
# Defining our class, which is derived from the above imported

class UserEncoder(JSONEncoder):

    def default(self, object):
        # Same as before
        if isinstance(object, User):
            return {'name': object.name, 'age': object.age,
                    object.__class__.__name__: True}
        # This time, using the JSONEncoder default
        else:
            return JSONEncoder.default(self, object)

In [31]:
userJSON2 = json.dumps(user, cls=UserEncoder)
print(userJSON2)

{"name": "Max", "age": 27, "User": true}


In [32]:
# The encoder can also be used directly:
userJSON3 = UserEncoder().encode(user)

In [33]:
print(userJSON3)

{"name": "Max", "age": 27, "User": true}


### DECODING object back to Python:

In [34]:
user = json.loads(userJSON)

In [36]:
print(user) # Prints back in python

{'name': 'Max', 'age': 27, 'User': True}


In [37]:
type(user) # It is still just a dict, not a class object

dict

In [38]:
# Must write a custom decoding method to return to object
def decode_user(dictionary):
    # If the key we created when encoding from a class exists
    if User.__name__ in dictionary:
        return User(name=dictionary['name'], age=dictionary['age'])

    # If the key is not there, just return the dict
    return dictionary

In [39]:
user2 = json.loads(userJSON, object_hook=decode_user)
print(type(user2))
print(user2.name)

<class '__main__.User'>
Max
